In [10]:
#Import libraries
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

%matplotlib inline

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import pandas as pd

#Read input data
train_data = pd.read_csv('/content/drive/MyDrive/ML/titanic-ml-project/data/processed/train.csv')
X_test = pd.read_csv('/content/drive/MyDrive/ML/titanic-ml-project/data/processed/test.csv')


In [16]:
# Define features and target variable
X_train = train_data.drop('Survived', axis=1)
Y_train = train_data['Survived']

## Logistic Regression

In [17]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=16, max_iter=500)
lr.fit(X_train, Y_train)

LogisticRegression(max_iter=500, random_state=16)

In [18]:
Y_pred = lr.predict(X_test)
Y_pred

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [19]:
from sklearn.ensemble import HistGradientBoostingClassifier

# Initialize and fit the model
model = HistGradientBoostingClassifier()
model.fit(X_train, Y_train)  # Assuming you have X_train and Y_train
Y_pred = model.predict(X_test)


In [20]:
diff_df = pd.DataFrame({'Actual' : Y_test, 'Predicted': Y_pred})
diff_df

NameError: name 'Y_test' is not defined

In [ ]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(Y_test, Y_pred)
conf_matrix

 **Observations:**<br>
 **True Negatives (TN) = 121**<br>
The model correctly predicted 121 people as "No Diabetes."<br>
 **False Positives (FP) = 30**<br>
The model incorrectly predicted 30 people as having diabetes when they actually didn't (Type I error).<br>
 **False Negatives (FN) = 31**<br>
The model incorrectly predicted 31 people as "No Diabetes" when they actually had diabetes (Type II error).<br>
 **True Positives (TP) = 49**<br>
The model correctly predicted 49 people as having diabetes.<br>

In [ ]:
from sklearn.metrics import classification_report

target_names = ['without diabetes', 'with diabetes']
print(classification_report(Y_test, Y_pred, target_names=target_names))